## Naive Bayes

In [1]:
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score, roc_auc_score
import numpy as np
from sklearn.preprocessing import MaxAbsScaler
from scipy.sparse import csr_matrix, vstack, hstack
from sklearn.model_selection import GridSearchCV

### Data Prep for ML

In [2]:
train = pd.read_pickle("data/train.pkl").dropna()
test = pd.read_pickle("data/test.pkl").dropna()

In [3]:
#One hot encoding
encoded_train = pd.get_dummies(train, columns = ['weighted_parent_sentiment_score','weighted_comment_sentiment_score'], drop_first=True)
encoded_test = pd.get_dummies(test, columns = ['weighted_parent_sentiment_score','weighted_comment_sentiment_score'], drop_first=True)

In [4]:
train_parent_tdidf_csr = vstack(encoded_train["parent_comment_tdidf"])
test_parent_tdidf_csr = vstack(encoded_test["parent_comment_tdidf"])

train_tdidf_csr = vstack(encoded_train["comment_tdidf"])
test_tdidf_csr = vstack(encoded_test["comment_tdidf"])

train_parent_bow_csr = vstack(encoded_train["parent_comment_bow"])
test_parent_bow_csr = vstack(encoded_test["parent_comment_bow"])

train_bow_csr = vstack(encoded_train["comment_bow"])
test_bow_csr = vstack(encoded_test["comment_bow"])

In [5]:
encoded_train.columns

Index(['comment', 'author', 'subreddit', 'score', 'ups', 'downs', 'date',
       'created_utc', 'parent_comment', 'comment_tokens',
       'parent_comment_tokens', 'comment_score', 'parent_comment_score',
       'comment_word_count', 'parent_comment_word_count',
       'comment_token_count', 'parent_comment_token_count',
       'comment_unique_word_count', 'parent_comment_unique_word_count',
       'comment_unique_token_count', 'parent_comment_unique_token_count',
       'comment_stopword_count', 'parent_comment_stopword_count',
       'comment_mean_word_length', 'parent_comment_mean_word_length',
       'comment_mean_token_length', 'parent_comment_mean_token_length',
       'comment_char_count', 'parent_comment_char_count',
       'comment_punctuation_count', 'parent_comment_punctuation_count',
       'comment_hashtag_count', 'parent_comment_hashtag_count',
       'comment_number_count', 'parent_comment_number_count', 'comment_bow',
       'parent_comment_bow', 'comment_tdidf', 'paren

In [33]:
# Remove score, ups, downs as Naive Bayes does not handle negative values
list_of_features = [
       'comment_word_count','parent_comment_word_count',
       'comment_token_count', 'parent_comment_token_count',
       'comment_unique_word_count', 'parent_comment_unique_word_count',
       'comment_unique_token_count', 'parent_comment_unique_token_count',
       'comment_stopword_count', 'parent_comment_stopword_count',
       'comment_mean_word_length', 'parent_comment_mean_word_length',
       'comment_mean_token_length', 'parent_comment_mean_token_length',
       'comment_char_count', 'parent_comment_char_count',
       'comment_punctuation_count', 'parent_comment_punctuation_count',
       'comment_hashtag_count', 'parent_comment_hashtag_count',
       'comment_number_count', 'parent_comment_number_count',
       'weighted_parent_sentiment_score_neutral',
       'weighted_parent_sentiment_score_positive',
       'weighted_comment_sentiment_score_neutral',
       'weighted_comment_sentiment_score_positive']

In [34]:
X_train_gen_features = csr_matrix(encoded_train[list_of_features])
y_train_LR = encoded_train['label']

X_test_gen_features = csr_matrix(encoded_test[list_of_features])
y_test_LR = encoded_test['label']

### Baseline 1: General Features

In [35]:
#Data Specific Prep
X_train_LR = X_train_gen_features


#scaler = MaxAbsScaler()
#X_train_LR = scaler.fit_transform(X_train_LR)


In [36]:
k = 5

model = MultinomialNB()
kf = KFold(n_splits=k, shuffle=True, random_state=42)

cross_val_scores = []
for train_index, val_index in kf.split(X_train_LR): 
    X_train, X_val = X_train_LR[train_index], X_train_LR[val_index]
    y_train, y_val = y_train_LR.iloc[train_index,], y_train_LR.iloc[val_index,]
    
    # Train the model on the training data
    model.fit(X_train, y_train)
    
    # Make predictions on the validation data
    y_pred = model.predict(X_val)
    
    # Calculate roc_auc and store it in the list
    roc_auc = roc_auc_score(y_val, y_pred)
    cross_val_scores.append(roc_auc)

mean_roc_auc = sum(cross_val_scores) / k
std_roc_auc = np.std(cross_val_scores)

print(f"Mean roc_auc: {mean_roc_auc}")
print(f"Standard Deviation of roc_auc: {std_roc_auc}")

Mean roc_auc: 0.526674402774953
Standard Deviation of roc_auc: 0.007576235749135441


### Baseline 2: Comment BoW

In [40]:
#Data Specific Prep
X_train_LR = train_bow_csr

#scaler = MaxAbsScaler()
#X_train_LR = scaler.fit_transform(X_train_LR)

In [41]:
k = 5

model = MultinomialNB()
kf = KFold(n_splits=k, shuffle=True, random_state=42)

cross_val_scores = []
for train_index, val_index in kf.split(X_train_LR): 
    X_train, X_val = X_train_LR[train_index], X_train_LR[val_index]
    y_train, y_val = y_train_LR.iloc[train_index,], y_train_LR.iloc[val_index,]
    
    # Train the model on the training data
    model.fit(X_train, y_train)
    
    # Make predictions on the validation data
    y_pred = model.predict(X_val)
    
    # Calculate roc_auc and store it in the list
    roc_auc = roc_auc_score(y_val, y_pred)
    cross_val_scores.append(roc_auc)

mean_roc_auc = sum(cross_val_scores) / k
std_roc_auc = np.std(cross_val_scores)

print(f"Mean roc_auc: {mean_roc_auc}")
print(f"Standard Deviation of roc_auc: {std_roc_auc}")

Mean roc_auc: 0.6424710012786635
Standard Deviation of roc_auc: 0.0022213151893920903


### Baseline 3: Comment TD-IDF

In [42]:
#Data Specific Prep
X_train_LR = train_tdidf_csr

#scaler = MaxAbsScaler()
#X_train_LR = scaler.fit_transform(X_train_LR)

In [43]:
k = 5

model = MultinomialNB()
kf = KFold(n_splits=k, shuffle=True, random_state=42)

cross_val_scores = []
for train_index, val_index in kf.split(X_train_LR): 
    X_train, X_val = X_train_LR[train_index], X_train_LR[val_index]
    y_train, y_val = y_train_LR.iloc[train_index,], y_train_LR.iloc[val_index,]
    
    # Train the model on the training data
    model.fit(X_train, y_train)
    
    # Make predictions on the validation data
    y_pred = model.predict(X_val)
    
    # Calculate roc_auc and store it in the list
    roc_auc = roc_auc_score(y_val, y_pred)
    cross_val_scores.append(roc_auc)

mean_roc_auc = sum(cross_val_scores) / k
std_roc_auc = np.std(cross_val_scores)

print(f"Mean roc_auc: {mean_roc_auc}")
print(f"Standard Deviation of roc_auc: {std_roc_auc}")

Mean roc_auc: 0.6370824287124377
Standard Deviation of roc_auc: 0.0018847132466299644


### Baseline 4: General Features + BoW

In [44]:
X_train_LR = hstack([X_train_gen_features,train_bow_csr])

#scaler = MaxAbsScaler()
#X_train_LR = scaler.fit_transform(X_train_LR)

In [45]:
k = 5

model = MultinomialNB()
kf = KFold(n_splits=k, shuffle=True, random_state=42)

cross_val_scores = []
for train_index, val_index in kf.split(X_train_LR): 
    X_train, X_val = X_train_LR[train_index], X_train_LR[val_index]
    y_train, y_val = y_train_LR.iloc[train_index,], y_train_LR.iloc[val_index,]
    
    # Train the model on the training data
    model.fit(X_train, y_train)
    
    # Make predictions on the validation data
    y_pred = model.predict(X_val)
    
    # Calculate roc_auc and store it in the list
    roc_auc = roc_auc_score(y_val, y_pred)
    cross_val_scores.append(roc_auc)

mean_roc_auc = sum(cross_val_scores) / k
std_roc_auc = np.std(cross_val_scores)

print(f"Mean roc_auc: {mean_roc_auc}")
print(f"Standard Deviation of roc_auc: {std_roc_auc}")

Mean roc_auc: 0.6519953387908837
Standard Deviation of roc_auc: 0.009362068345928452


### Baseline 5: General Features + TD-IDF

In [46]:
X_train_LR = hstack([X_train_gen_features,train_tdidf_csr])

scaler = MaxAbsScaler()
X_train_LR = scaler.fit_transform(X_train_LR)

In [47]:
k = 5

model = MultinomialNB()
kf = KFold(n_splits=k, shuffle=True, random_state=42)

cross_val_scores = []
for train_index, val_index in kf.split(X_train_LR): 
    X_train, X_val = X_train_LR[train_index], X_train_LR[val_index]
    y_train, y_val = y_train_LR.iloc[train_index,], y_train_LR.iloc[val_index,]
    
    # Train the model on the training data
    model.fit(X_train, y_train)
    
    # Make predictions on the validation data
    y_pred = model.predict(X_val)
    
    # Calculate roc_auc and store it in the list
    roc_auc = roc_auc_score(y_val, y_pred)
    cross_val_scores.append(roc_auc)

mean_roc_auc = sum(cross_val_scores) / k
std_roc_auc = np.std(cross_val_scores)

print(f"Mean roc_auc: {mean_roc_auc}")
print(f"Standard Deviation of roc_auc: {std_roc_auc}")

Mean roc_auc: 0.6454446664795951
Standard Deviation of roc_auc: 0.0013280918014056543


### Baseline 6: Gen Features + Comment TDIDF + Parent TDIDF

In [48]:
X_train_LR = hstack([X_train_gen_features,train_tdidf_csr,train_parent_tdidf_csr])

scaler = MaxAbsScaler()
X_train_LR = scaler.fit_transform(X_train_LR)

In [49]:
k = 5

model = MultinomialNB()
kf = KFold(n_splits=k, shuffle=True, random_state=42)

cross_val_scores = []
for train_index, val_index in kf.split(X_train_LR): 
    X_train, X_val = X_train_LR[train_index], X_train_LR[val_index]
    y_train, y_val = y_train_LR.iloc[train_index,], y_train_LR.iloc[val_index,]
    
    # Train the model on the training data
    model.fit(X_train, y_train)
    
    # Make predictions on the validation data
    y_pred = model.predict(X_val)
    
    # Calculate roc_auc and store it in the list
    roc_auc = roc_auc_score(y_val, y_pred)
    cross_val_scores.append(roc_auc)

mean_roc_auc = sum(cross_val_scores) / k
std_roc_auc = np.std(cross_val_scores)

print(f"Mean roc_auc: {mean_roc_auc}")
print(f"Standard Deviation of roc_auc: {std_roc_auc}")

Mean roc_auc: 0.6390059846116081
Standard Deviation of roc_auc: 0.0018672700910615002


### Baseline 7: Gen + Parent TDIDF

In [50]:
X_train_LR = hstack([X_train_gen_features,train_parent_tdidf_csr])

scaler = MaxAbsScaler()
X_train_LR = scaler.fit_transform(X_train_LR)

In [51]:
k = 5

model = MultinomialNB()
kf = KFold(n_splits=k, shuffle=True, random_state=42)

cross_val_scores = []
for train_index, val_index in kf.split(X_train_LR): 
    X_train, X_val = X_train_LR[train_index], X_train_LR[val_index]
    y_train, y_val = y_train_LR.iloc[train_index,], y_train_LR.iloc[val_index,]
    
    # Train the model on the training data
    model.fit(X_train, y_train)
    
    # Make predictions on the validation data
    y_pred = model.predict(X_val)
    
    # Calculate roc_auc and store it in the list
    roc_auc = roc_auc_score(y_val, y_pred)
    cross_val_scores.append(roc_auc)

mean_roc_auc = sum(cross_val_scores) / k
std_roc_auc = np.std(cross_val_scores)

print(f"Mean roc_auc: {mean_roc_auc}")
print(f"Standard Deviation of roc_auc: {std_roc_auc}")

Mean roc_auc: 0.5660865867859248
Standard Deviation of roc_auc: 0.0010944462801385922


### Hyper Parameter Tuning with Grid Search CV

In [55]:
### Using gen features and tdidf
X_train_LR = hstack([X_train_gen_features,train_bow_csr])

#scaler = MaxAbsScaler()
#X_train_LR = scaler.fit_transform(X_train_LR)

In [64]:
param_grid = {
    'alpha': [0.2, 0.4, 0.6, 0.8, 1.0],  # You can extend this list
    'fit_prior': [True, False],
}


In [65]:
grid_search = GridSearchCV(MultinomialNB(), param_grid, cv=5, scoring='roc_auc', verbose=4)
grid_search.fit(X_train_LR, y_train_LR)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END .........alpha=0.2, fit_prior=True;, score=0.674 total time=   0.0s
[CV 2/5] END .........alpha=0.2, fit_prior=True;, score=0.677 total time=   0.0s
[CV 3/5] END .........alpha=0.2, fit_prior=True;, score=0.673 total time=   0.0s
[CV 4/5] END .........alpha=0.2, fit_prior=True;, score=0.675 total time=   0.0s
[CV 5/5] END .........alpha=0.2, fit_prior=True;, score=0.678 total time=   0.0s
[CV 1/5] END ........alpha=0.2, fit_prior=False;, score=0.674 total time=   0.0s
[CV 2/5] END ........alpha=0.2, fit_prior=False;, score=0.677 total time=   0.0s
[CV 3/5] END ........alpha=0.2, fit_prior=False;, score=0.673 total time=   0.0s
[CV 4/5] END ........alpha=0.2, fit_prior=False;, score=0.675 total time=   0.0s
[CV 5/5] END ........alpha=0.2, fit_prior=False;, score=0.678 total time=   0.0s
[CV 1/5] END .........alpha=0.4, fit_prior=True;, score=0.678 total time=   0.0s
[CV 2/5] END .........alpha=0.4, fit_prior=True;

GridSearchCV(cv=5, estimator=MultinomialNB(),
             param_grid={'alpha': [0.2, 0.4, 0.6, 0.8, 1.0],
                         'fit_prior': [True, False]},
             scoring='roc_auc', verbose=4)

In [66]:
best_params = grid_search.best_params_
print("Best Parameters:", best_params)

# Best score achieved during grid search
best_score = grid_search.best_score_
print("Best Score:", best_score)

Best Parameters: {'alpha': 1.0, 'fit_prior': True}
Best Score: 0.6868055819138096


In [67]:
# Best estimator (the fitted model with the best parameters)
best_estimator = grid_search.best_estimator_
print("Best Estimator:", best_estimator)

# Results for all parameter combinations
cv_results = grid_search.cv_results_

Best Estimator: MultinomialNB()


In [69]:
mean_test_scores = cv_results['mean_test_score']
std_test_scores = cv_results['std_test_score']
params = cv_results['params']

pd.set_option('display.max_colwidth',None)
results_df = pd.DataFrame({'Params':params,'Mean Score':mean_test_scores,'STD':std_test_scores})
results_df["Params"] = results_df["Params"].apply(lambda x: ', '.join([f'{key}: {value}' for key, value in x.items()]))
results_df.sort_values('STD', inplace=True)
results_df.sort_values('Mean Score', ascending=False,inplace=True)
results_df


#for mean_score, std_score, param in zip(mean_test_scores, std_test_scores, params):
    #print(f"Mean Score: {mean_score:.3f} (±{std_score:.3f}) for params: {param}")

,Params,Mean Score,STD
8,"alpha: 1.0, fit_prior: True",0.686806,0.001614
9,"alpha: 1.0, fit_prior: False",0.686806,0.001614
6,"alpha: 0.8, fit_prior: True",0.684786,0.001658
7,"alpha: 0.8, fit_prior: False",0.684786,0.001658
4,"alpha: 0.6, fit_prior: True",0.682340,0.001728
5,"alpha: 0.6, fit_prior: False",0.682340,0.001728
2,"alpha: 0.4, fit_prior: True",0.679275,0.001796
3,"alpha: 0.4, fit_prior: False",0.679275,0.001796
0,"alpha: 0.2, fit_prior: True",0.675122,0.001910
1,"alpha: 0.2, fit_prior: False",0.675122,0.001910


### Compare to Test Set

In [62]:
X_test_LR = hstack([X_test_gen_features,test_bow_csr])
#X_test_LR = scaler.fit_transform(X_test_LR)

In [63]:
y_pred_test = best_estimator.predict(X_test_LR)
roc_auc_test = roc_auc_score(y_test_LR, y_pred_test)
print(f"roc_auc test: {roc_auc_test}")

roc_auc test: 0.5854166377891767
